# Error Detection and Debugging Notebook
Systematic error detection, debugging, and resolution

This notebook focuses on:
- Error detection across all components
- Debugging workflows and tools
- Performance bottlenecks identification
- Code quality analysis
- Resolution strategies

Following engineering principles: systematic debugging, root cause analysis, preventive measures

In [ ]:
# Setup environment and comprehensive imports
import os
import sys
import asyncio
import traceback
import inspect
from pathlib import Path
from typing import Dict, Any, List, Optional
from datetime import datetime
import logging

# Add app to path for imports
sys.path.append(str(Path("..").resolve()))

# Core imports for comprehensive testing
from app.core.config import get_settings
from app.core.logging import get_logger, setup_logging
from app.core.exceptions import AgentError, ToolError

# Setup enhanced logging for debugging
setup_logging()
logger = get_logger("notebook_debug")

# Set more verbose logging for debugging
logging.getLogger().setLevel(logging.DEBUG)

logger.info("🐛 Starting comprehensive error detection and debugging")

In [ ]:
# Debug 1: Configuration and Environment Validation
def debug_configuration():
    """Comprehensive configuration and environment debugging."""
    
    logger.info("🔍 Debugging configuration and environment...")
    
    issues_found = []
    recommendations = []
    
    try:
        # Test settings loading
        settings = get_settings()
        logger.info("✅ Settings loaded successfully")
        
        # Validate critical settings
        critical_settings = [
            ("GOOGLE_API_KEY", "Google API key for LLM"),
            ("DATABASE_URL", "Database connection"),
            ("ENVIRONMENT", "Environment configuration")
        ]
        
        for setting, description in critical_settings:
            if hasattr(settings, setting.lower()) or setting in os.environ:
                logger.info(f"✅ {description} configured")
            else:
                issue = f"❌ Missing {description} ({setting})"
                issues_found.append(issue)
                recommendations.append(f"Set {setting} environment variable")
                logger.warning(issue)
        
        # Test LLM configuration
        try:
            gemini_config = settings.gemini_config
            required_keys = ["model_name", "temperature", "max_output_tokens", "api_key"]
            
            for key in required_keys:
                if key not in gemini_config or not gemini_config[key]:
                    issue = f"❌ Missing Gemini config: {key}"
                    issues_found.append(issue)
                    logger.warning(issue)
                else:
                    logger.info(f"✅ Gemini {key} configured")
                    
        except Exception as e:
            issue = f"❌ Gemini configuration error: {e}"
            issues_found.append(issue)
            recommendations.append("Check Gemini LLM configuration in settings")
            logger.error(issue)
        
        # Test database configuration
        try:
            db_url = getattr(settings, 'database_url', os.getenv('DATABASE_URL'))
            if db_url:
                logger.info("✅ Database URL configured")
                
                # Test database connection
                import psycopg2
                try:
                    # Parse database URL or use default connection
                    if db_url.startswith('postgresql://'):
                        conn = psycopg2.connect(db_url)
                    else:
                        # Use default Docker connection
                        conn = psycopg2.connect(
                            host="postgres",
                            database="react_agent_db",
                            user="agent_user", 
                            password="agent_pass",
                            port="5432"
                        )
                    conn.close()
                    logger.info("✅ Database connection successful")
                except Exception as db_e:
                    issue = f"❌ Database connection failed: {db_e}"
                    issues_found.append(issue)
                    recommendations.append("Ensure PostgreSQL container is running (make up)")
                    logger.error(issue)
            else:
                issue = "❌ Database URL not configured"
                issues_found.append(issue)
                recommendations.append("Set DATABASE_URL or ensure Docker environment")
                logger.warning(issue)
                
        except Exception as e:
            issue = f"❌ Database configuration error: {e}"
            issues_found.append(issue)
            logger.error(issue)
    
    except Exception as e:
        issue = f"❌ Critical configuration error: {e}"
        issues_found.append(issue)
        recommendations.append("Check core configuration setup")
        logger.error(issue)
        logger.error(traceback.format_exc())
    
    return {
        "issues_found": issues_found,
        "recommendations": recommendations,
        "status": "healthy" if not issues_found else "issues_detected"
    }

# Run configuration debugging
config_debug_result = debug_configuration()
print(f"Configuration status: {config_debug_result['status']}")
if config_debug_result['issues_found']:
    print("Issues found:")
    for issue in config_debug_result['issues_found']:
        print(f"  {issue}")
    print("Recommendations:")
    for rec in config_debug_result['recommendations']:
        print(f"  - {rec}")

In [ ]:
# Debug 2: Import and Dependency Validation
def debug_imports_and_dependencies():
    """Debug import issues and dependency problems."""
    
    logger.info("🔍 Debugging imports and dependencies...")
    
    import_issues = []
    dependency_status = {}
    
    # Critical dependencies to test
    critical_imports = [
        ("langchain", "LangChain core"),
        ("langgraph", "LangGraph framework"),
        ("langchain_google_genai", "Google Gemini integration"),
        ("streamlit", "Web interface"),
        ("matplotlib", "Plotting"),
        ("plotly", "Interactive plotting"),
        ("psycopg2", "PostgreSQL driver"),
        ("sympy", "Symbolic mathematics"),
        ("numpy", "Numerical computing"),
        ("pandas", "Data manipulation")
    ]
    
    for module_name, description in critical_imports:
        try:
            __import__(module_name)
            dependency_status[module_name] = "✅ Available"
            logger.info(f"✅ {description} ({module_name}) imported successfully")
        except ImportError as e:
            issue = f"❌ Failed to import {description} ({module_name}): {e}"
            import_issues.append(issue)
            dependency_status[module_name] = f"❌ Failed: {str(e)}"
            logger.error(issue)
        except Exception as e:
            issue = f"❌ Unexpected error importing {description} ({module_name}): {e}"
            import_issues.append(issue)
            dependency_status[module_name] = f"❌ Error: {str(e)}"
            logger.error(issue)
    
    # Test app module imports
    app_modules = [
        ("app.core.config", "Core configuration"),
        ("app.agents.graph", "Agent graph"),
        ("app.agents.state", "Agent state"),
        ("app.tools.registry", "Tool registry"),
        ("app.tools.integral_tool", "Integral tool"),
        ("app.tools.plot_tool", "Plot tool"),
        ("app.tools.analysis_tool", "Analysis tool"),
        ("app.models.agent_state", "Agent state model"),
        ("app.ui.components.chat", "Chat component")
    ]
    
    for module_name, description in app_modules:
        try:
            __import__(module_name)
            dependency_status[module_name] = "✅ Available"
            logger.info(f"✅ {description} ({module_name}) imported successfully")
        except ImportError as e:
            issue = f"❌ Failed to import {description} ({module_name}): {e}"
            import_issues.append(issue)
            dependency_status[module_name] = f"❌ Failed: {str(e)}"
            logger.error(issue)
        except Exception as e:
            issue = f"❌ Unexpected error importing {description} ({module_name}): {e}"
            import_issues.append(issue)
            dependency_status[module_name] = f"❌ Error: {str(e)}"
            logger.error(issue)
    
    # Check Python version compatibility
    python_version = sys.version_info
    if python_version.major == 3 and python_version.minor >= 8:
        logger.info(f"✅ Python version {python_version.major}.{python_version.minor} is compatible")
    else:
        issue = f"❌ Python version {python_version.major}.{python_version.minor} may not be compatible (requires 3.8+)"
        import_issues.append(issue)
        logger.warning(issue)
    
    return {
        "import_issues": import_issues,
        "dependency_status": dependency_status,
        "python_version": f"{python_version.major}.{python_version.minor}.{python_version.micro}",
        "status": "healthy" if not import_issues else "issues_detected"
    }

# Run import debugging
import_debug_result = debug_imports_and_dependencies()
print(f"Import status: {import_debug_result['status']}")
print(f"Python version: {import_debug_result['python_version']}")

if import_debug_result['import_issues']:
    print("\nImport issues found:")
    for issue in import_debug_result['import_issues']:
        print(f"  {issue}")

In [ ]:
# Debug 3: Component Integration Issues
async def debug_component_integration():
    """Debug integration issues between components."""
    
    logger.info("🔍 Debugging component integration...")
    
    integration_issues = []
    component_status = {}
    
    # Test 1: Tool Integration
    try:
        logger.info("Testing tool integration...")
        from app.tools.registry import ToolRegistry
        from app.tools.integral_tool import IntegralTool
        
        # Create registry and tool
        registry = ToolRegistry()
        integral_tool = IntegralTool()
        
        # Test tool registration
        registry.register_tool(
            integral_tool,
            categories=["mathematical"],
            tags=["integration"]
        )
        
        # Test tool retrieval
        retrieved_tool = registry.get_tool("integral_calculator")
        if retrieved_tool:
            component_status["tool_registry"] = "✅ Working"
            logger.info("✅ Tool registry integration working")
        else:
            issue = "❌ Tool registry: tool not found after registration"
            integration_issues.append(issue)
            component_status["tool_registry"] = "❌ Failed"
            logger.error(issue)
            
    except Exception as e:
        issue = f"❌ Tool integration error: {e}"
        integration_issues.append(issue)
        component_status["tool_registry"] = f"❌ Error: {str(e)}"
        logger.error(issue)
        logger.error(traceback.format_exc())
    
    # Test 2: Agent State Integration
    try:
        logger.info("Testing agent state integration...")
        from app.agents.state import MathAgentState, WorkflowSteps, WorkflowStatus
        from uuid import uuid4
        
        # Create test state
        test_state = MathAgentState(
            messages=[],
            conversation_id=uuid4(),
            session_id="debug_test",
            user_id="debug_user",
            created_at=datetime.now(),
            updated_at=datetime.now(),
            current_step=WorkflowSteps.PROBLEM_ANALYSIS,
            iteration_count=0,
            max_iterations=10,
            workflow_status=WorkflowStatus.ACTIVE,
            user_input="Test integration",
            problem_type="integration",
            reasoning_trace=[],
            tool_calls=[],
            final_result=None,
            error_info=None,
            memory=None,
            visualization_data=None,
            metadata={}
        )
        
        # Validate state structure
        if test_state["conversation_id"] and test_state["session_id"]:
            component_status["agent_state"] = "✅ Working"
            logger.info("✅ Agent state integration working")
        else:
            issue = "❌ Agent state: invalid state structure"
            integration_issues.append(issue)
            component_status["agent_state"] = "❌ Failed"
            logger.error(issue)
            
    except Exception as e:
        issue = f"❌ Agent state integration error: {e}"
        integration_issues.append(issue)
        component_status["agent_state"] = f"❌ Error: {str(e)}"
        logger.error(issue)
        logger.error(traceback.format_exc())
    
    # Test 3: LLM Integration
    try:
        logger.info("Testing LLM integration...")
        from langchain_google_genai import ChatGoogleGenerativeAI
        from app.core.config import get_settings
        
        settings = get_settings()
        gemini_config = settings.gemini_config
        
        # Create LLM instance
        llm = ChatGoogleGenerativeAI(
            model=gemini_config["model_name"],
            temperature=gemini_config["temperature"],
            max_output_tokens=gemini_config["max_output_tokens"],
            google_api_key=gemini_config["api_key"]
        )
        
        # Test simple invocation
        response = await asyncio.wait_for(
            llm.ainvoke("Test connection. Respond with 'OK' only."),
            timeout=10.0
        )
        
        if response and hasattr(response, 'content'):
            component_status["llm_integration"] = "✅ Working"
            logger.info(f"✅ LLM integration working: {response.content[:50]}...")
        else:
            issue = "❌ LLM integration: no valid response"
            integration_issues.append(issue)
            component_status["llm_integration"] = "❌ Failed"
            logger.error(issue)
            
    except asyncio.TimeoutError:
        issue = "❌ LLM integration: timeout (network/API issue)"
        integration_issues.append(issue)
        component_status["llm_integration"] = "❌ Timeout"
        logger.error(issue)
    except Exception as e:
        issue = f"❌ LLM integration error: {e}"
        integration_issues.append(issue)
        component_status["llm_integration"] = f"❌ Error: {str(e)}"
        logger.error(issue)
        logger.error(traceback.format_exc())
    
    # Test 4: Database Integration
    try:
        logger.info("Testing database integration...")
        from app.database.connection import get_connection
        
        # Test database connection
        conn = await get_connection()
        if conn:
            await conn.close()
            component_status["database"] = "✅ Working"
            logger.info("✅ Database integration working")
        else:
            issue = "❌ Database integration: connection failed"
            integration_issues.append(issue)
            component_status["database"] = "❌ Failed"
            logger.error(issue)
            
    except Exception as e:
        issue = f"❌ Database integration error: {e}"
        integration_issues.append(issue)
        component_status["database"] = f"❌ Error: {str(e)}"
        logger.error(issue)
    
    return {
        "integration_issues": integration_issues,
        "component_status": component_status,
        "status": "healthy" if not integration_issues else "issues_detected"
    }

# Run component integration debugging
integration_debug_result = await debug_component_integration()
print(f"Integration status: {integration_debug_result['status']}")
print("\nComponent status:")
for component, status in integration_debug_result['component_status'].items():
    print(f"  {component}: {status}")

if integration_debug_result['integration_issues']:
    print("\nIntegration issues:")
    for issue in integration_debug_result['integration_issues']:
        print(f"  {issue}")

In [ ]:
# Debug 4: Workflow Execution Issues
async def debug_workflow_execution():
    """Debug workflow execution and identify bottlenecks."""
    
    logger.info("🔍 Debugging workflow execution...")
    
    workflow_issues = []
    execution_metrics = {}
    
    try:
        # Test workflow creation
        logger.info("Testing workflow creation...")
        from app.agents.graph import create_mathematical_workflow
        
        start_time = datetime.now()
        workflow = await create_mathematical_workflow()
        creation_time = (datetime.now() - start_time).total_seconds()
        
        execution_metrics["workflow_creation_time"] = creation_time
        logger.info(f"✅ Workflow created in {creation_time:.2f}s")
        
        # Test simple workflow execution
        logger.info("Testing simple workflow execution...")
        from app.agents.state import MathAgentState, WorkflowSteps, WorkflowStatus
        from uuid import uuid4
        
        test_state = MathAgentState(
            messages=[],
            conversation_id=uuid4(),
            session_id="debug_workflow",
            user_id="debug_user",
            created_at=datetime.now(),
            updated_at=datetime.now(),
            current_step=WorkflowSteps.PROBLEM_ANALYSIS,
            iteration_count=0,
            max_iterations=5,  # Reduced for debugging
            workflow_status=WorkflowStatus.ACTIVE,
            user_input="Calculate integral of x^2",
            problem_type=None,
            reasoning_trace=[],
            tool_calls=[],
            final_result=None,
            error_info=None,
            memory=None,
            visualization_data=None,
            metadata={"debug": True}
        )
        
        # Execute workflow with timeout and monitoring
        execution_start = datetime.now()
        step_count = 0
        last_state = None
        execution_trace = []
        
        try:
            async for state in workflow.astream(test_state):
                step_count += 1
                current_step = state.get("current_step", "unknown")
                step_time = datetime.now()
                
                execution_trace.append({
                    "step": step_count,
                    "node": current_step,
                    "timestamp": step_time.isoformat(),
                    "status": state.get("workflow_status", "unknown")
                })
                
                logger.info(f"📍 Debug Step {step_count}: {current_step}")
                
                # Check for infinite loops or excessive iterations
                if step_count > 15:
                    issue = f"❌ Workflow execution: excessive iterations ({step_count})"
                    workflow_issues.append(issue)
                    logger.warning(issue)
                    break
                
                # Check for stuck states
                if last_state and last_state.get("current_step") == current_step:
                    issue = f"❌ Workflow execution: stuck in state {current_step}"
                    workflow_issues.append(issue)
                    logger.warning(issue)
                
                last_state = state
            
            execution_time = (datetime.now() - execution_start).total_seconds()
            execution_metrics["workflow_execution_time"] = execution_time
            execution_metrics["steps_executed"] = step_count
            execution_metrics["execution_trace"] = execution_trace
            
            # Validate final state
            if last_state:
                final_status = last_state.get("workflow_status", "unknown")
                has_result = last_state.get("final_result") is not None
                has_errors = last_state.get("error_info") is not None
                
                execution_metrics["final_status"] = final_status
                execution_metrics["has_result"] = has_result
                execution_metrics["has_errors"] = has_errors
                
                if final_status in [WorkflowStatus.COMPLETED, "completed"]:
                    logger.info(f"✅ Workflow completed successfully in {execution_time:.2f}s")
                elif has_errors:
                    issue = f"❌ Workflow completed with errors: {last_state.get('error_info')}"
                    workflow_issues.append(issue)
                    logger.warning(issue)
                else:
                    issue = f"⚠️ Workflow ended with status: {final_status}"
                    workflow_issues.append(issue)
                    logger.warning(issue)
            else:
                issue = "❌ Workflow execution: no final state"
                workflow_issues.append(issue)
                logger.error(issue)
        
        except asyncio.TimeoutError:
            issue = "❌ Workflow execution: timeout"
            workflow_issues.append(issue)
            logger.error(issue)
        except Exception as e:
            issue = f"❌ Workflow execution error: {e}"
            workflow_issues.append(issue)
            logger.error(issue)
            logger.error(traceback.format_exc())
    
    except Exception as e:
        issue = f"❌ Workflow creation error: {e}"
        workflow_issues.append(issue)
        logger.error(issue)
        logger.error(traceback.format_exc())
    
    return {
        "workflow_issues": workflow_issues,
        "execution_metrics": execution_metrics,
        "status": "healthy" if not workflow_issues else "issues_detected"
    }

# Run workflow debugging
workflow_debug_result = await debug_workflow_execution()
print(f"Workflow status: {workflow_debug_result['status']}")
print(f"Execution metrics: {workflow_debug_result['execution_metrics']}")

if workflow_debug_result['workflow_issues']:
    print("\nWorkflow issues:")
    for issue in workflow_debug_result['workflow_issues']:
        print(f"  {issue}")

In [ ]:
# Debug 5: Performance and Resource Analysis
import psutil
import gc
import time

def debug_performance_and_resources():
    """Analyze performance and resource usage patterns."""
    
    logger.info("📈 Debugging performance and resources...")
    
    performance_issues = []
    resource_metrics = {}
    
    # Get system information
    process = psutil.Process()
    
    # Memory analysis
    memory_info = process.memory_info()
    memory_percent = process.memory_percent()
    
    resource_metrics["memory_usage_mb"] = memory_info.rss / 1024 / 1024
    resource_metrics["memory_percent"] = memory_percent
    
    # CPU analysis
    cpu_percent = process.cpu_percent(interval=1)
    resource_metrics["cpu_percent"] = cpu_percent
    
    # Disk usage
    disk_usage = psutil.disk_usage('/')
    resource_metrics["disk_usage_percent"] = (disk_usage.used / disk_usage.total) * 100
    
    logger.info(f"Memory usage: {resource_metrics['memory_usage_mb']:.1f}MB ({memory_percent:.1f}%)")
    logger.info(f"CPU usage: {cpu_percent:.1f}%")
    logger.info(f"Disk usage: {resource_metrics['disk_usage_percent']:.1f}%")
    
    # Check for performance issues
    if memory_percent > 80:
        issue = f"❌ High memory usage: {memory_percent:.1f}%"
        performance_issues.append(issue)
        logger.warning(issue)
    
    if cpu_percent > 90:
        issue = f"❌ High CPU usage: {cpu_percent:.1f}%"
        performance_issues.append(issue)
        logger.warning(issue)
    
    if resource_metrics["disk_usage_percent"] > 90:
        issue = f"❌ High disk usage: {resource_metrics['disk_usage_percent']:.1f}%"
        performance_issues.append(issue)
        logger.warning(issue)
    
    # Memory leak detection
    gc.collect()  # Force garbage collection
    
    # Check for large objects in memory
    import sys
    large_objects = []
    for obj in gc.get_objects():
        size = sys.getsizeof(obj)
        if size > 1024 * 1024:  # Objects larger than 1MB
            large_objects.append({
                "type": type(obj).__name__,
                "size_mb": size / 1024 / 1024
            })
    
    if large_objects:
        resource_metrics["large_objects"] = large_objects[:5]  # Top 5
        logger.info(f"Found {len(large_objects)} large objects in memory")
        for obj in large_objects[:3]:
            logger.info(f"  {obj['type']}: {obj['size_mb']:.1f}MB")
    
    # Thread analysis
    thread_count = process.num_threads()
    resource_metrics["thread_count"] = thread_count
    
    if thread_count > 50:
        issue = f"❌ High thread count: {thread_count}"
        performance_issues.append(issue)
        logger.warning(issue)
    
    # File descriptor analysis (Unix systems)
    try:
        fd_count = process.num_fds()
        resource_metrics["file_descriptors"] = fd_count
        
        if fd_count > 1000:
            issue = f"❌ High file descriptor count: {fd_count}"
            performance_issues.append(issue)
            logger.warning(issue)
    except AttributeError:
        # Windows doesn't have num_fds
        pass
    
    return {
        "performance_issues": performance_issues,
        "resource_metrics": resource_metrics,
        "status": "healthy" if not performance_issues else "issues_detected"
    }

# Run performance debugging
performance_debug_result = debug_performance_and_resources()
print(f"Performance status: {performance_debug_result['status']}")
print(f"Memory usage: {performance_debug_result['resource_metrics']['memory_usage_mb']:.1f}MB")
print(f"CPU usage: {performance_debug_result['resource_metrics']['cpu_percent']:.1f}%")

if performance_debug_result['performance_issues']:
    print("\nPerformance issues:")
    for issue in performance_debug_result['performance_issues']:
        print(f"  {issue}")

In [ ]:
# Debug Summary and Resolution Plan
def generate_debug_summary():
    """Generate comprehensive debug summary and resolution plan."""
    
    logger.info("📋 Generating debug summary and resolution plan...")
    
    # Collect all debug results
    all_results = {
        "configuration": config_debug_result,
        "imports": import_debug_result,
        "integration": integration_debug_result,
        "workflow": workflow_debug_result,
        "performance": performance_debug_result
    }
    
    # Categorize issues by severity
    critical_issues = []
    warning_issues = []
    performance_issues = []
    
    for category, result in all_results.items():
        if result.get("status") == "issues_detected":
            for issue_key in ["issues_found", "import_issues", "integration_issues", "workflow_issues", "performance_issues"]:
                issues = result.get(issue_key, [])
                for issue in issues:
                    if "❌" in issue:
                        if "timeout" in issue.lower() or "connection" in issue.lower():
                            critical_issues.append(f"[{category.upper()}] {issue}")
                        elif "performance" in issue.lower() or "memory" in issue.lower():
                            performance_issues.append(f"[{category.upper()}] {issue}")
                        else:
                            critical_issues.append(f"[{category.upper()}] {issue}")
                    elif "⚠️" in issue:
                        warning_issues.append(f"[{category.upper()}] {issue}")
    
    # Generate resolution plan
    resolution_plan = []
    
    # Critical issues first
    if critical_issues:
        resolution_plan.append("🚨 CRITICAL ISSUES (Fix Immediately):")
        for issue in critical_issues:
            resolution_plan.append(f"  {issue}")
            
            # Add specific recommendations
            if "database" in issue.lower():
                resolution_plan.append("    → Run 'make up' to start PostgreSQL container")
            elif "api" in issue.lower() or "google" in issue.lower():
                resolution_plan.append("    → Check GOOGLE_API_KEY environment variable")
            elif "import" in issue.lower():
                resolution_plan.append("    → Install missing dependencies with 'poetry install'")
            elif "timeout" in issue.lower():
                resolution_plan.append("    → Check network connectivity and API quotas")
    
    # Warning issues
    if warning_issues:
        resolution_plan.append("\n⚠️  WARNING ISSUES (Monitor and Fix):")
        for issue in warning_issues:
            resolution_plan.append(f"  {issue}")
    
    # Performance issues
    if performance_issues:
        resolution_plan.append("\n📈 PERFORMANCE ISSUES (Optimize):")
        for issue in performance_issues:
            resolution_plan.append(f"  {issue}")
    
    # Overall system health
    healthy_components = sum(1 for result in all_results.values() if result.get("status") == "healthy")
    total_components = len(all_results)
    health_percentage = (healthy_components / total_components) * 100
    
    # Recommendations based on health
    if health_percentage >= 80:
        system_status = "🟢 HEALTHY"
        resolution_plan.append(f"\n{system_status} - System is functioning well ({health_percentage:.0f}% components healthy)")
    elif health_percentage >= 60:
        system_status = "🟡 DEGRADED"
        resolution_plan.append(f"\n{system_status} - Some issues detected ({health_percentage:.0f}% components healthy)")
        resolution_plan.append("  → Focus on critical issues first")
    else:
        system_status = "🔴 CRITICAL"
        resolution_plan.append(f"\n{system_status} - Major issues detected ({health_percentage:.0f}% components healthy)")
        resolution_plan.append("  → Requires immediate attention")
    
    # Next steps
    resolution_plan.append("\n📝 RECOMMENDED NEXT STEPS:")
    resolution_plan.append("1. Fix critical issues in order of appearance")
    resolution_plan.append("2. Verify fixes by running relevant test notebooks")
    resolution_plan.append("3. Monitor performance metrics")
    resolution_plan.append("4. Run full integration tests")
    resolution_plan.append("5. Update documentation with findings")
    
    summary = {
        "system_status": system_status,
        "health_percentage": health_percentage,
        "critical_issues_count": len(critical_issues),
        "warning_issues_count": len(warning_issues),
        "performance_issues_count": len(performance_issues),
        "resolution_plan": resolution_plan,
        "detailed_results": all_results
    }
    
    logger.info(f"🎯 Debug summary generated - System status: {system_status}")
    
    return summary

# Generate final summary
debug_summary = generate_debug_summary()

print("="*80)
print("🐛 COMPREHENSIVE DEBUG SUMMARY")
print("="*80)
print(f"System Status: {debug_summary['system_status']}")
print(f"Health: {debug_summary['health_percentage']:.0f}%")
print(f"Critical Issues: {debug_summary['critical_issues_count']}")
print(f"Warnings: {debug_summary['warning_issues_count']}")
print(f"Performance Issues: {debug_summary['performance_issues_count']}")
print("\n" + "\n".join(debug_summary['resolution_plan']))

## Error Detection and Debugging Results

This notebook performed comprehensive error detection and debugging across all system components:

### System Health Overview
- **Configuration**: Environment and settings validation
- **Dependencies**: Import and module availability  
- **Integration**: Component interaction testing
- **Workflow**: End-to-end execution validation
- **Performance**: Resource usage and optimization

### Debug Categories Tested:
1. ✅ **Configuration Validation**: Environment variables, API keys, settings
2. ✅ **Import Analysis**: Dependency availability and compatibility
3. ✅ **Integration Testing**: Component communication and data flow
4. ✅ **Workflow Execution**: End-to-end process validation
5. ✅ **Performance Analysis**: Resource usage and bottlenecks

### Key Debugging Features:
- 🔍 **Systematic Error Detection**: Comprehensive component scanning
- 🛠️ **Root Cause Analysis**: Detailed error tracking with context
- 📊 **Performance Monitoring**: Resource usage tracking
- 🎯 **Targeted Recommendations**: Specific fix suggestions
- 📋 **Resolution Planning**: Prioritized action items

### Resolution Strategy:
1. **Critical Issues First**: Database, API, core functionality
2. **Warning Issues**: Non-blocking but important fixes
3. **Performance Issues**: Optimization opportunities
4. **Preventive Measures**: Monitoring and alerting setup

### Debugging Tools Used:
- Exception handling and tracebacks
- Resource monitoring (CPU, memory, disk)
- Component isolation testing
- Integration validation
- Performance profiling

This systematic approach ensures all issues are identified, categorized, and resolved efficiently while maintaining code quality and system reliability.